In [ ]:
# Mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# We use this library to move the data in colab environment
import shutil 

# Load the train dataset
# Specify the source file path in your Colab environment
source_file_path = '/content/drive/MyDrive/DataChallenge_augmentedFiles/train345.zip'

# Specify the destination file path in your Google Drive
destination_file_path = '/content/train345.zip'

# Copy the file from the source path to the destination path
shutil.copyfile(source_file_path, destination_file_path)

# load the validation dataset
source_file_path = '/content/drive/MyDrive/DataChallenge_augmentedFiles/val.zip'
# Specify the destination file path in your Google Drive
destination_file_path = '/content/val.zip'

# Copy the file from the source path to the destination path
shutil.copyfile(source_file_path, destination_file_path)




'/content/val.zip'

In [ ]:
# Unzip the datasets
# !unzip train345.zip
# !unzip '/content/Vague.zip' -d "/content/add_data"
!unzip val

Archive:  val.zip
   creating: val/
   creating: val/OK/
  inflating: val/OK/CH214FC2_328057_0_RESPONSIVITE_3.png  
  inflating: val/OK/CH214FC2_328088_0_RESPONSIVITE_3.png  
  inflating: val/OK/CH214FC2_328722_0_RESPONSIVITE_3.png  
  inflating: val/OK/CH214FC2_329289_0_RESPONSIVITE_3.png  
  inflating: val/OK/CH214FC2_329313_0_RESPONSIVITE_3.png  
  inflating: val/OK/CH214FC2_329318_0_RESPONSIVITE_3.png  
  inflating: val/OK/CH214FC2_329320_0_RESPONSIVITE_3.png  
  inflating: val/OK/CH214FC2_329330_0_RESPONSIVITE_3.png  
  inflating: val/OK/CH214FC2_330354_0_RESPONSIVITE_3.png  
  inflating: val/OK/CH214FC2_330360_0_RESPONSIVITE_3.png  
  inflating: val/OK/CH214FC2_330612_0_RESPONSIVITE_3.png  
  inflating: val/OK/CH214FC2_330994_0_RESPONSIVITE_3.png  
  inflating: val/OK/CH214FC2_331005_0_RESPONSIVITE_3.png  
  inflating: val/OK/CH214FC2_331017_0_RESPONSIVITE_3.png  
  inflating: val/OK/CH214FC2_331334_0_RESPONSIVITE_3.png  
  inflating: val/OK/CH214FC2_331643_1_RESPONSIVITE_3.png  

In [ ]:
# loading the test dataset

source_file_path = '//content/drive/MyDrive/Test_files/dataset_2.zip'
# Specify the destination file path in your Google Drive
destination_file_path = '/content/dataset_2.zip'

# Copy the file from the source path to the destination path
shutil.copyfile(source_file_path, destination_file_path)

'/content/dataset_2.zip'

In [ ]:
!unzip dataset_2.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset_2/CH214P_366861_0_RESPONSIVITE_3.png  
  inflating: __MACOSX/dataset_2/._CH214P_366861_0_RESPONSIVITE_3.png  
  inflating: dataset_2/CH214G_346490_0_RESPONSIVITE_3.png  
  inflating: __MACOSX/dataset_2/._CH214G_346490_0_RESPONSIVITE_3.png  
  inflating: dataset_2/CH214G_347282_1_RESPONSIVITE_3.png  
  inflating: __MACOSX/dataset_2/._CH214G_347282_1_RESPONSIVITE_3.png  
  inflating: dataset_2/CH214FC3_352780_0_RESPONSIVITE_3.png  
  inflating: __MACOSX/dataset_2/._CH214FC3_352780_0_RESPONSIVITE_3.png  
  inflating: dataset_2/CH214FC3_362004_1_RESPONSIVITE_3.png  
  inflating: __MACOSX/dataset_2/._CH214FC3_362004_1_RESPONSIVITE_3.png  
  inflating: dataset_2/CH214L_337146_0_RESPONSIVITE_3.png  
  inflating: __MACOSX/dataset_2/._CH214L_337146_0_RESPONSIVITE_3.png  
  inflating: dataset_2/CH214L_330387_0_RESPONSIVITE_3.png  
  inflating: __MACOSX/dataset_2/._CH214L_330387_0_RESPONSIVITE_3.png  
  inflating: dataset_2/C

In [ ]:
# Importing libraries
import numpy as np
import os
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications import ResNet101 # We used vgg16,vgg19,resnet50,inceptionV3 and mobilenet but Resnet101 is the best
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import Sequential, layers
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.layers import Rescaling
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Lambda

In [ ]:

# Load the pre-trained ResNet50 model
resnet = ResNet101(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the pre-trained layers
for layer in resnet.layers:
    layer.trainable = False

# Since our dataset is grayscale images and the model is pretrained on RGB images we use this function to convert them to RGB and then pass them through the model
# Convert grayscale to RGB 
def grayscale_to_rgb(image):
    return tf.image.grayscale_to_rgb(image)

# Add new layers for your own dataset
inputs = tf.keras.Input(shape=(img_height, img_width, 1))
rgb_images = Lambda(grayscale_to_rgb)(inputs)
x = resnet(rgb_images)
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.25)(x)
predictions = Dense(2, activation='softmax')(x)

# Create a new model
model = Model(inputs=inputs, outputs=predictions)
opt = tf.keras.optimizers.experimental.AdamW()


# This function manages the learning rate of the model and minimizes it as epoch gets bigger
def scheduler(epoch, lr):
   if epoch < 5:
     return lr
   else:
     return lr * tf.math.exp(-0.1)
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
# Compile the model
model.compile(optimizer= 'adam', loss='categorical_crossentropy', 
              metrics=['accuracy',
              keras.metrics.TruePositives(name='tp'),
              keras.metrics.FalsePositives(name='fp'),
              keras.metrics.TrueNegatives(name='tn'),
              keras.metrics.FalseNegatives(name='fn'), 
              keras.metrics.Precision(name='precision', class_id = 1),
              keras.metrics.Recall(name='recall', class_id= 1)])





171446536/171446536 [==============================] - 1s 0us/step


In [ ]:
# loading the weights
model.load_weights("model_checkpoint_24.h5")

In [ ]:
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 512, 640, 1)]     0         
                                                                 
 lambda (Lambda)             (None, 512, 640, 3)       0         
                                                                 
 resnet101 (Functional)      (None, 16, 20, 2048)      42658176  
                                                                 
 flatten (Flatten)           (None, 655360)            0         
                                                                 
 dense (Dense)               (None, 1024)              671089664 
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800

In [ ]:
from tensorflow import keras
from keras import Sequential, layers
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Rescaling
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.utils import load_img, img_to_array
import glob
import os
from keras.preprocessing import image
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
# Load your trained model
# model_path = r"D:\DATACHALLENGE\Data_challenge\Models\Set_1\resnet50_v2.h5"
# df = pd.read_csv(r'/content/drive/MyDrive/Test_files/dataset_2_test.csv')

# # model = load_model(model_path)
# print(model.summary())

# Function to load and preprocess new images
def load_image(filename):
    img_height = 512
    img_width = 640
    img = load_img(filename, color_mode="grayscale", interpolation='bilinear', target_size=(img_height, img_width))

    # FOR TRANSFER LEARNING
    # img = load_img(filename, interpolation='bilinear', target_size=(img_height, img_width))
    
    img = img_to_array(img)
    img_for_plot = img.copy()
    img = np.expand_dims(img, axis=0)  # Add a batch dimension
    return img, img_for_plot

class_labels = ["OK", "Vague"]  # Update with your actual class labels


VAGUE_PATH = r"/content/val/OK/*.png"
n_false = 0
paths = glob.glob(VAGUE_PATH)
# print(paths)

file_names = [os.path.basename(path) for path in paths]
# print(file_names)
for j, p in enumerate(paths):
    # Path to your new image
    # Load and preprocess the new image

    new_image = load_image(p)[0]

    # Perform predictions on the new image
    predictions = model.predict(new_image)
    class_index = np.argmax(predictions[0])

    # Interpret the predictions
    predicted_class = class_labels[class_index]
    # Write in corresponding row of df
    # df.Label[df.name == file_names[j]] = predicted_class

    if predicted_class == 'Vague':
        n_false+=1
        # plt.figure()
        # plt.imshow(load_image(p)[1], cmap = 'gray')
    print(f"Image Number : {j} Predicted class:", predicted_class)

print('FALSE VAGUE CASES :', n_false)
# Save the updated DataFrame as a CSV file
# df.to_csv(r'/content/drive/MyDrive/Test_files/Set1.csv', index=False)

1/1 [==============================] - 1s 1s/step
Image Number : 0 Predicted class: OK
1/1 [==============================] - 1s 1s/step
Image Number : 1 Predicted class: OK
1/1 [==============================] - 1s 1s/step
Image Number : 2 Predicted class: OK
1/1 [==============================] - 1s 1s/step
Image Number : 3 Predicted class: OK
1/1 [==============================] - 2s 2s/step
Image Number : 4 Predicted class: OK
1/1 [==============================] - 2s 2s/step
Image Number : 5 Predicted class: OK
1/1 [==============================] - 1s 1s/step
Image Number : 6 Predicted class: OK
1/1 [==============================] - 1s 1s/step
Image Number : 7 Predicted class: OK
1/1 [==============================] - 1s 1s/step
Image Number : 8 Predicted class: OK
1/1 [==============================] - 1s 1s/step
Image Number : 9 Predicted class: OK
1/1 [==============================] - 1s 1s/step
Image Number : 10 Predicted class: OK
1/1 [==============================] - 1s 